**Instituto de Informática - UFG**

Disciplina: Inteligência Computacional

Prof. Cedric Luiz de Carvalho


Grupo ( J )

---


Analista de dados:
* Arthur Cavalcante de Andrade
* William Teixeira Pires Junior

---

# Solução para o Problema

O problema remete à previsão do óbito de pacientes internados.

Dessa forma, iremos implementar uma rede neural que seja capaz de prever se um paciente que foi internado em um hospital irá morrer ou não.

#### Carregando as bibliotecas e módulos necessários

In [3]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

#### Lendo os dados e realizando verificações básicas de dados

In [4]:
df = pd.read_csv('statistic_data.csv') 
print(df.shape)
df.describe().transpose()

(91713, 77)


,count,mean,std,min,25%,50%,75%,max
age,91713.0,62.295247,16.384012,16.000000,53.00000,64.000000,75.000000,89.00000
bmi,91713.0,29.185818,8.118970,14.844926,23.78776,28.007812,32.653061,67.81499
elective_surgery,91713.0,0.183736,0.387271,0.000000,0.00000,0.000000,0.000000,1.00000
ethnicity,91713.0,0.635395,1.369270,0.000000,0.00000,0.000000,0.000000,5.00000
height,91713.0,169.641588,10.716578,137.200000,162.56000,170.000000,177.800000,195.59000
...,...,...,...,...,...,...,...,...
leukemia,91713.0,0.007011,0.083438,0.000000,0.00000,0.000000,0.000000,1.00000
lymphoma,91713.0,0.004100,0.063898,0.000000,0.00000,0.000000,0.000000,1.00000
solid_tumor_with_metastasis,91713.0,0.020477,0.141626,0.000000,0.00000,0.000000,0.000000,1.00000
apache_3j_bodysystem,91713.0,4.168777,3.809760,0.000000,0.00000,5.000000,8.000000,10.00000


Observando o resumo da variável `hospital_death`, observamos que o valor médio é 0.086, o que significa que cerca de 8.6% das observações no conjunto de dados têm diabetes.

Visando uma boa proporção nas amostras, podemos considerar fazer uma rede neural com dados de teste que possuem a quantidade de mortes e não mortes com proporções equivalentes, e outra rede neural com amostras aleatórias. Em seguida, comparar os resultados.

#### Criando os conjuntos de dados de treinamento e teste

##### Dados aleatórios

Inicialmente dividimos a variável de destino e o resto do conjunto de dados, criamos arrays das variáveis independente e dependentes, e em seguida separamos dados de treinamento e de teste.

In [10]:
#divide a variável de destino e o resto do conjunto de dados
target_column = ['hospital_death']
predictors = list(set(list(df.columns))-set(target_column))

#cria arrays de variáveis independentes (X) e dependentes (Y)
X = df[predictors].values
y = df[target_column].values

#divide os dados em conjunto de dados de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)

#imprime a forma dos dados de treinamento e teste
print(X_train.shape); print(X_test.shape)

(64199, 76)
(27514, 76)


##### Dados com proporção equivalente

#### Construindo, Prevendo e Avaliando o Modelo de Rede Neural

Nesta etapa, construiremos o modelo de rede neural usando o objeto estimador da biblioteca scikit-learn, **Multi-Layer Perceptron Classifier**.

A primeira linha instancia o modelo com o argumento `hidden_layer_sizes` definido como três camadas, que tem o mesmo número de neurônios que a contagem de recursos no conjunto de dados. 

A segunda linha ajusta o modelo de dados ao treinamento, enquanto a terceira e quarta linhas usam o modelo treinado para gerar previsões no conjunto de dados de treinamento e teste, respectivamente.

In [11]:
mlp = MLPClassifier(hidden_layer_sizes=(76,76,76), activation='relu', solver='adam', max_iter=500)
mlp.fit(X_train,y_train)

predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)

C:\Users\TECNO\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Uma vez que as previsões são geradas, podemos avaliar o desempenho do modelo.

Sendo um algoritmo de classificação, primeiro importaremos os módulos necessários, o que é feito na primeira linha do código abaixo.

A segunda e terceira linhas de código imprimem a matriz de confusão e os resultados do relatório de confusão nos dados de treinamento.

In [12]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train,predict_train))

[[58265   375]
 [ 1038  4521]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     58640
           1       0.92      0.81      0.86      5559

    accuracy                           0.98     64199
   macro avg       0.95      0.90      0.93     64199
weighted avg       0.98      0.98      0.98     64199



O próximo passo é avaliar o desempenho do modelo nos dados de teste que é feito com as linhas de código abaixo.

In [13]:
print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test,predict_test))

[[23996  1162]
 [ 1495   861]]
              precision    recall  f1-score   support

           0       0.94      0.95      0.95     25158
           1       0.43      0.37      0.39      2356

    accuracy                           0.90     27514
   macro avg       0.68      0.66      0.67     27514
weighted avg       0.90      0.90      0.90     27514

